In [ ]:
# 기사 목록

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import time

# 크롬 드라이버 옵션 설정
chrome_options = Options()

chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--blink-settings=imagesEnabled=false")  # 이미지 로드 비활성화
chrome_options.add_argument("--disable-extensions")  # 확장 프로그램 비활성화
chrome_options.add_argument("--disable-gpu")  # GPU 가속 비활성화

# Keywords to search
keywords = [
    "검색어",
]
# 검색 기간 설정
start_date = "2024-01-01"
end_date = "2024-08-31"    


def fetch_articles(search_term, page_number, start_date, end_date):
    """지정된 검색어와 날짜 범위로 중앙일보에서 기사 스크래핑."""
    search_url = (
        f"https://www.joongang.co.kr/search/news?"
        f"keyword={search_term}&startDate={start_date}&endDate={end_date}&"
        f"sfield=all&page={page_number}"
    )
    
    driver.get(search_url)

    try:
        # 명시적인 대기
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.story_list .card'))
        )
    except:
        # 요소가 없으면 빈 리스트 반환
        return []

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    articles = []

    for item in soup.select('.story_list .card'):
        title = item.select_one('.headline').text.strip() if item.select_one('.headline') else ''
        url = item.select_one('a')['href'] if item.select_one('a') else ''
        source = item.select_one('.meta .source').text.strip() if item.select_one('.meta .source') else ''
        date = item.select_one('.meta .date').text.strip().split(' ')[0] if item.select_one('.meta .date') else ''

        articles.append({
            "media": source,
            "title": title,
            "date": date,
            "section": '',
            "article_type": '',
            'writer': '',
            "page": '',
            "url": url,
            'content': '',
            "articleId": ''
        })

    return articles

def save_to_excel(all_articles, filename):
    """수집한 기사 데이터를 엑셀 파일로 저장."""
    df = pd.DataFrame(all_articles)
    df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
    df.to_excel(filename, index=False)

# 각 검색어에 대해 결과 가져오기
all_articles = []
seen_urls = set()  # 중복 확인을 위한 URL 집합

try:
    # 웹 드라이버 초기화
    driver = webdriver.Chrome(options=chrome_options)

    for term in keywords:
        page_number = 1  # 페이지 번호 설정
        while True:
            print(f"Fetching articles for '{term}' - Page {page_number}")
            articles = fetch_articles(term, page_number, start_date, end_date)

            # 새로운 기사인지 확인하고 중복 제거
            new_articles = [article for article in articles if article['url'] not in seen_urls]

            # 새로운 기사가 없으면 종료
            if not new_articles:
                print(f"No more new articles for '{term}' at page {page_number}. Moving to next keyword.")
                break

            # 새로운 기사 URL을 seen_urls에 추가
            seen_urls.update(article['url'] for article in new_articles)
            all_articles.extend(new_articles)
            page_number += 1

finally:
    # 기사 제목과 링크 저장
    filename = f"1중앙일보 {start_date}~{end_date}.xlsx"
    save_to_excel(all_articles, filename)

    # 드라이버 종료
    driver.quit()

# 결과 확인을 위해 DataFrame 출력 (선택사항)
print(pd.DataFrame(all_articles))



In [ ]:
# 본문 저장sssss

import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# 크롬 드라이버 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--blink-settings=imagesEnabled=false")  # 이미지 로드 비활성화
chrome_options.add_argument("--disable-extensions")  # 확장 프로그램 비활성화
chrome_options.add_argument("--disable-gpu")  # GPU 가속 비활성화

# 1. 출처와 파일명 변수 설정
file_name = '파일명'  # 파일명 설정
base_path = rf'경로'  # 경로 설정

# 2. 임시 파일 경로와 최종 파일 경로 생성
temp_output_path = os.path.join(base_path, f'{file_name}.xlsx')
# 임시 파일 에러를 대비해 본 파일에 덮어쓴다.
output_path = os.path.join(base_path, f'{file_name}_본문.xlsx')  # 최종 파일 경로
error_log_path = os.path.join(base_path, f'{file_name}_error_log.txt')  # 에러 로그 파일 경로

# 3. 파일 경로 생성 및 엑셀 파일 불러오기
input_file = os.path.join(base_path, f'{file_name}.xlsx')
df = pd.read_excel(input_file, engine='openpyxl')

# 웹 드라이버 초기화
driver = webdriver.Chrome(options=chrome_options)

def fetch_article_details(article_url, row):
    if (pd.notna(row['content']) and row['content'] != "") and (pd.notna(row['sub-title']) and row['sub-title'] != ""):
        print(f"이미 수집된 기사입니다. URL 접근을 건너뜁니다: {article_url}")
        return "", ""
    
    driver.get(article_url)
    
    try:        
        # 기사 본문을 가져오기 위해 대기
        WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#article_body'))
        )
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 부제 추출 (div.ab_sub_headingline)
        subtitle = ""
        subtitle_tag = soup.find('div', class_='ab_sub_headingline')
        subtitle = subtitle_tag.get_text(separator='\n', strip=True) if subtitle_tag else ''

        # 제거할 태그들 삭제
        unwanted_tags = ['div#ad_art_content_btm', 'p.caption', 'div.series_list_wrap', 'div.player_area', 'div#ad_art_content_mid', 'div.ab_photo photo_center', 'body.template_special', 'div.html_photo_center', 'div.html_photo', 'div.divTable']
        for tag in unwanted_tags:
            for unwanted in soup.select(tag):
                unwanted.decompose()

        content = "\n".join([p.get_text(separator='\n', strip=True) for p in soup.select('#article_body')])  
        if content.startswith('='):
            content = "'" + content
    except:
        content = ''

    return content, subtitle

error_log = []

def update_articles_with_details(input_df):
    for index, row in input_df.iterrows():

        print(f"Fetching details for article {index + 1}/{len(input_df)}")
        content, subtitle = fetch_article_details(row['url'], row)
        
        # 기존 DataFrame에 새로운 값 업데이트
        input_df.at[index, 'content'] = content
        input_df.at[index, 'sub-title'] = subtitle

        # 100개마다 파일 저장
        if (index + 1) % 100 == 0:
            df.to_excel(temp_output_path, index=False)
            print(f"임시 파일을 저장했습니다: {temp_output_path}")

    driver.quit()   
    return input_df

# 기사 링크에 들어가서 본문을 원본 DataFrame에 추가
updated_df = update_articles_with_details(df)

# 엑셀 파일로 저장
updated_df = updated_df[~updated_df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
updated_df = updated_df[~updated_df['content'].str.contains('블락비|나인뮤지스', case=False, na=False)]
updated_df.to_excel(output_path, index=False)

# 에러 로그 파일 저장
if error_log:
    with open(error_log_path, 'w') as f:
        for url in error_log:
            f.write(f"{url}\n")
    print(f"에러 로그가 저장되었습니다: {error_log_path}")


In [ ]:
import os
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from openpyxl.utils.exceptions import IllegalCharacterError  # 올바른 임포트 경로

# 크롬 드라이버 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--blink-settings=imagesEnabled=false")  # 이미지 로드 비활성화
chrome_options.add_argument("--disable-extensions")  # 확장 프로그램 비활성화
chrome_options.add_argument("--disable-gpu")  # GPU 가속 비활성화

# 1. 출처와 파일명 변수 설정
file_name = '파일명'  # 파일명 설정
base_path = rf'경로'  # 경로 설정

# 2. 임시 파일 경로와 최종 파일 경로 생성
temp_output_path = os.path.join(base_path, f'{file_name}_temp.xlsx')
output_path = os.path.join(base_path, f'{file_name}_본문.xlsx')  # 최종 파일 경로
error_log_path = os.path.join(base_path, f'{file_name}_error_log.txt')  # 에러 로그 파일 경로

# 3. 파일 경로 생성 및 엑셀 파일 불러오기
input_file = os.path.join(base_path, f'{file_name}.xlsx')
df = pd.read_excel(input_file, engine='openpyxl')

# 웹 드라이버 초기화
driver = webdriver.Chrome(options=chrome_options)

# 에러 로그 초기화
error_log = []

def remove_illegal_characters(text):
    if isinstance(text, str):
        # Excel에서 허용하지 않는 유니코드 제어 문자 패턴
        illegal_char_pattern = re.compile(
            r'[\x00-\x08\x0B\x0C\x0E-\x1F]'
        )
        return illegal_char_pattern.sub('', text)
    return text

def fetch_article_details(article_url):
    """
    기사 URL을 받아 본문과 부제를 추출하는 함수.
    """
    try:
        driver.get(article_url)
        
        # 기사 본문을 가져오기 위해 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#article_body'))
        )
        soup = BeautifulSoup(driver.page_source, 'html.parser')

                # 부제 추출 (div.ab_sub_headingline)
        subtitle = ""
        subtitle_tag = soup.find('div', class_='ab_sub_headingline')
        if subtitle_tag:
            subtitle = subtitle_tag.get_text(separator='\n', strip=True)
            subtitle_tag.decompose() 
        else:
            subtitle_tag = soup.find('div', class_='ab_subtitle')
            if subtitle_tag:
                subtitle = subtitle_tag.get_text(separator='\n', strip=True)
                subtitle_tag.decompose()         

        # 제거할 태그들 삭제
        unwanted_tags = [
            'div#ad_art_content_btm', 
            'p.caption', 
            'div.series_list_wrap', 
            'div.player_area', 
            'div#ad_art_content_mid', 
            'div.ab_photo.photo_center', 
            'body.template_special',
            'div.ab_related_article',
            'section.related_link',
            'div.divTable'
        ]
        for tag in unwanted_tags:
            for unwanted in soup.select(tag):
                unwanted.decompose()

        # 본문 추출
        content = "\n".join([p.get_text(separator='\n', strip=True) for p in soup.select('#article_body')])  
        if content.startswith('='):
            content = "'" + content

        # 불법 문자 제거
        content = remove_illegal_characters(content)
        subtitle = remove_illegal_characters(subtitle)

        return content, subtitle
    except Exception as e:
        print(f"Error fetching {article_url}: {e}")
        error_log.append(article_url)
        return '', ''

def update_articles_with_details(input_df):
    """
    DataFrame을 순회하며 기사 상세 정보를 업데이트하는 함수.
    """
    try:
        for index, row in input_df.iterrows():
            # # 'content' 컬럼이 이미 채워져 있는 경우 건너뜀
            # if pd.notna(row['content']) and row['content'].strip() != "":
            #     print(f"이미 수집된 기사입니다. URL 접근을 건너뜁니다: {row['url']}")
            #     continue

            print(f"Fetching details for article {index + 1}/{len(input_df)}: {row['url']}")
            content, subtitle = fetch_article_details(row['url'])
            
            # 기존 DataFrame에 새로운 값 업데이트
            input_df.at[index, 'content'] = content
            input_df.at[index, 'sub-title'] = subtitle

            # 100개마다 파일 저장
            if (index + 1) % 100 == 0:
                try:
                    # 임시 파일 저장 전에 불법 문자 제거
                    input_df.to_excel(temp_output_path, index=False)
                    print(f"임시 파일을 저장했습니다: {temp_output_path}")
                except IllegalCharacterError as e:
                    print(f"임시 파일 저장 중 불법 문자가 발견되었습니다: {e}")
                except Exception as e:
                    print(f"임시 파일 저장 중 다른 오류가 발생했습니다: {e}")
    finally:
        driver.quit()
    return input_df

# 기사 링크에 들어가서 본문을 원본 DataFrame에 추가
updated_df = update_articles_with_details(df)

# 불법 문자 제거 (추가적으로 안전을 위해 모든 객체형 컬럼에 적용)
object_columns = updated_df.select_dtypes(['object']).columns
for col in object_columns:
    updated_df[col] = updated_df[col].apply(remove_illegal_characters)

# 엑셀 파일로 저장
try:
    # 특정 키워드가 포함된 기사를 필터링
    filtered_df = updated_df[
        ~updated_df['title'].str.contains('블락비|나인뮤지스', case=False, na=False) &
        ~updated_df['content'].str.contains('블락비|나인뮤지스', case=False, na=False)
    ]
    filtered_df.to_excel(output_path, index=False)
    print(f"최종 파일이 저장되었습니다: {output_path}")
except IllegalCharacterError as e:
    print(f"엑셀 저장 중 불법 문자가 발견되었습니다: {e}")
except Exception as e:
    print(f"엑셀 저장 중 다른 오류가 발생했습니다: {e}")

# 에러 로그 파일 저장
if error_log:
    with open(error_log_path, 'w', encoding='utf-8') as f:
        for url in error_log:
            f.write(f"{url}\n")
    print(f"에러 로그가 저장되었습니다: {error_log_path}")
